In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as col
from matplotlib.colors import LogNorm
from datetime import date, timedelta,datetime
import calendar
from isoweek import Week
import seaborn as sns
from stop_words import get_stop_words
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

sns.set_style("whitegrid")
sns.set_context("poster")


from  config import *

In [3]:
authors = pd.read_csv(DATA_PATH+"authors_id.csv",index_col=0)
authors=authors[['id','displayName']]
df = pd.read_csv(DATA_PATH+"all_content.csv")
stop_words_en = get_stop_words('en')

In [4]:
df = pd.merge(df,authors,left_on='author_id',right_on='id',how='inner')

In [5]:
df=df[['displayName','content','collection_id','created_at']]
df.columns = ['author_name','content','collection_id','created_at']

In [6]:
df.head(1)

,author_name,content,collection_id,created_at
0,Kiev,NATO has already agreed (at 2pm EU time) to he...,81858342,1236267047


In [7]:
df1 = df[df.author_name=="Njegos"]

In [8]:
content1 =df1.content.str.cat()


In [19]:
content2 = "@Goldsack The question now is whether the cure is worse than the disease.@RandomCommenter7 "
words = re.split(r'\W+', content2.lower())
words = filter(lambda x: x.isalpha(), words)
words

['goldsack',
 'the',
 'question',
 'now',
 'is',
 'whether',
 'the',
 'cure',
 'is',
 'worse',
 'than',
 'the',
 'disease']

In [29]:
def word_count(df,author_name):
    df1 = df[df.author_name==author_name]
    content =df1.content.str.cat()
    words = re.split(r'\W+', content.lower())
    words = filter(lambda x: x.isalpha(), words)
    words = [word for word in words 
    if len(word) > 3 and word not in stop_words_en and word  not in authors.displayName.tolist()]
    
    word_count = Counter(words)
    return word_count


In [30]:
author_list =['Iron Knee', 'christopher c','Francheska (Maya) Smith-Johnson',
 'Njegos',
 'maljoffre',
 'Paul A. Myers',
 "Paul Munton's Potimarron",
 'TylerDurden',
 'Harold Godwinson',
 'Ealing',
 'MarkGB',
 'rrrahul',
 'Harry Lime',
 'Gulliver',
 'Good European',
 'Judyw',
 'Don10984408',
 'Apostle',
 'RiskAdjustedReturn',
 'Ric']


In [ ]:
word_df = pd.DataFrame()
for author in author_list:
    word_counter = word_count(df,author)
    word_count_df = pd.DataFrame.from_dict(word_counter,
                                     orient='index')
#     file_df.columns = [fname.replace('input/papers/', '').replace('.txt', '')]
    word_count_df.ix[:, 0] = word_count_df.values.flatten() / float(word_count_df.values.sum())
    word_count_df.columns=[author]
    word_df = pd.merge(word_df,word_count_df,left_index=True,right_index=True,how='outer')

word_df = word_df.fillna(0)
print "Number of unique words: %s" % len(word_df)
word_df.head(10)

In [ ]:
U, sigma, V = np.linalg.svd(word_df)

In [ ]:

v_df = pd.DataFrame(V, columns=word_df.columns)
v_df.apply(lambda x: np.round(x, decimals=2))

In [ ]:
plt.imshow(V, interpolation='none')
ax = plt.gca()
plt.xticks(xrange(len(v_df.columns.values)))
plt.yticks(xrange(len(v_df.index.values)))
plt.title("$V$")
ax.set_xticklabels(v_df.columns.values, rotation=90)
plt.colorbar();